In [1]:
import os, csv, re, nltk
# import preprocessor as p


In [2]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


In [3]:
train_path = os.path.join('/home/rajesh/PycharmProjects/FIRE2019/HASOC/Extracted_dataset/english_dataset', 'english_dataset.tsv')
print(os.path.abspath(train_path))

eng_stopwords = set(nltk.corpus.stopwords.words("english"))

tknzr = nltk.TweetTokenizer(strip_handles=True)

text, label1, label2, label3 = [], [], [], []
with open(os.path.abspath(train_path), 'r', encoding='utf-8')as f:
    line_count = 0
    data = csv.reader(f, delimiter='\t', )
    for row in data:
        if line_count == 0:
            pass
        else:
            # text_input = remove_emoji(p.tokenize(p.clean(row[1])))
            text_input = tknzr.tokenize(row[1].strip())
            text_input = remove_emoji(" ".join([t for t in text_input if not t.startswith('http')]))
            text_input = tknzr.tokenize(text_input)
            text.append(" ".join([t for t in text_input if t not in eng_stopwords]))
            label1.append(row[2])
            label2.append(row[3])
            label3.append(row[4])
        line_count += 1



C:\home\rajesh\PycharmProjects\FIRE2019\HASOC\Extracted_dataset\english_dataset\english_dataset.tsv


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\home\\rajesh\\PycharmProjects\\FIRE2019\\HASOC\\Extracted_dataset\\english_dataset\\english_dataset.tsv'

In [ ]:
print(text[0])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# 
X_train, X_test, y_train, y_test = train_test_split(text, label1, test_size=0.2)


tfidf_transformer = TfidfVectorizer()
tfidf_transformer.fit_transform(text)

X_train_tfidf = tfidf_transformer.transform(X_train)
X_test_tfidf = tfidf_transformer.transform(X_test)

print(X_test_tfidf.shape)
print(X_train_tfidf.shape)

gnb = MultinomialNB()
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=50, random_state=42)
lr = LogisticRegression()
lin_svc = SVC(kernel='linear',probability=True)
model = XGBClassifier()


In [ ]:
gnb.fit(X_train_tfidf, y_train)
sgd.fit(X_train_tfidf, y_train)
lr.fit(X_train_tfidf, y_train)
lin_svc.fit(X_train_tfidf, y_train)
model.fit(X_train_tfidf, y_train)


pred_gnb = gnb.predict(X_test_tfidf)
pred_sgd = sgd.predict(X_test_tfidf)
pred_lr = lr.predict(X_test_tfidf)
pred_lin_svc = lin_svc.predict(X_test_tfidf)
pred_model = model.predict(X_test_tfidf)

# print("Multinomial Naive-Bayes accuracy : ",accuracy_score(y_test, pred_gnb, normalize = True))
# print('\n')
# print("Multinomial Naive-Bayes Classification Report : ",classification_report(y_test, pred_gnb))
# 
# print("SGD accuracy : ",accuracy_score(y_test, pred_sgd, normalize = True))
# print('\n')
# print("SGD Classification Report : ",classification_report(y_test, pred_sgd))
# 
# print("LR accuracy : ",accuracy_score(y_test, pred_lr, normalize = True))
# print('\n')
# print("LR Classification Report : ",classification_report(y_test, pred_lr))

print("LinSVC accuracy : ",accuracy_score(y_test, pred_lin_svc, normalize = True))
print('\n')
print("LinSVC Classification Report : ",classification_report(y_test, pred_lin_svc))

print("XGD accuracy : ",accuracy_score(y_test, pred_model, normalize = True))
print('\n')
print("XGD Classification Report : ",classification_report(y_test, pred_model))





In [ ]:
import pandas as pd
test_path = os.path.join('/home/rajesh/PycharmProjects/FIRE2019/HASOC/hasoc2019_en_test.tsv')

with open(test_path, 'r') as f1:
    total_lines = f1.readlines()

text_id_list = [t.split('\t')[0] for t in total_lines]
text__list = [t.split('\t')[1].strip('\n') for t in total_lines]

_text_list = []
for te in text__list:
    text_input = tknzr.tokenize(te.strip())
    text_input = remove_emoji(" ".join([t for t in text_input if not t.startswith('http')]))
    text_input = tknzr.tokenize(text_input)
    _text_list.append(" ".join([t for t in text_input if t not in eng_stopwords]))


In [ ]:
import numpy as np

X_test_ = tfidf_transformer.transform(np.array(_text_list))
xgd_predic_ = model.predict(X_test_)
svc_predic_ = lin_svc.predict(X_test_)

In [ ]:
print(xgd_predic_)
print(svc_predic_)

In [ ]:
with open('HASCO_level1_svc.csv', 'w') as f1:
    for idx, p in enumerate(svc_predic_):
        f1.write(text_id_list[idx]+','+str(p))
        f1.write('\n')